In [2]:
import spacy
import stanza

stanza.download('id')

nlpStanza = stanza.Pipeline('id')
nlpSpacy = spacy.load("en_core_web_sm")

2024-12-11 13:18:06 INFO: Downloaded file to C:\Users\Pongo\stanza_resources\resources.json
2024-12-11 13:18:06 INFO: Downloading default packages for language: id (Indonesian) ...
2024-12-11 13:18:07 INFO: File exists: C:\Users\Pongo\stanza_resources\id\default.zip
2024-12-11 13:18:09 INFO: Finished downloading models and saved to C:\Users\Pongo\stanza_resources
2024-12-11 13:18:09 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-12-11 13:18:10 INFO: Downloaded file to C:\Users\Pongo\stanza_resources\resources.json
2024-12-11 13:18:11 INFO: Loading these models for language: id (Indonesian):
| Processor    | Package      |
-------------------------------
| tokenize     | gsd          |
| mwt          | gsd          |
| pos          | gsd_charlm   |
| lemma        | gsd_nocharlm |
| constituency | icon_charlm  |
| depparse     | gsd_charlm   |

2024-12-11 13:18:11 INFO: Using device: cuda
2024-12-11 13:18:11 INFO: Loading: tokenize
c:\Users\Pongo\AppData\Local\Programs\Python\Python312\Lib\site-packages\stanza\models\tokenization\trainer.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, th

In [14]:
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from langdetect import detect, LangDetectException
from tqdm import tqdm
from joblib import dump

tqdm.pandas()

nltk.download('stopwords')
from nltk.corpus import stopwords

df = pd.read_csv('data.csv', encoding="utf-8", encoding_errors="replace")
stopwordsBahasa = pd.read_csv("stopwordbahasa.csv")

stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = str(text).lower() 
    text = ''.join([char for char in text if char.isalnum() or char.isspace()])
    text = ' '.join([word for word in text.split() if (word not in stop_words and word not in stopwordsBahasa.values)])

    # try:
    #     lang = detect(text)
    # except LangDetectException:
    #     lang = "en" 

    # if lang == "en":
    #     doc = nlpSpacy(text)
    #     finalText = ""
    #     i = 0
    #     while i < len(doc):
    #         if doc[i].pos_ == "ADJ" or doc[i].pos_ == "VERB":
    #             finalText = finalText + doc[i].text + " "
    #         elif doc[i].pos_ == "PART" or doc[i].pos_ == "ADV":
    #             finalText = finalText + doc[i].text + " "
    #         i += 1
    #     text = finalText
    # else:
    #     doc = nlpStanza(text)
    #     finalText = ""
    #     i = 0
    #     while i < len(doc.sentences[0].tokens):
    #         if doc.sentences[0].tokens[i].words[0].upos == "ADJ" or doc.sentences[0].tokens[i].words[0].upos == "VERB":
    #             finalText = finalText + doc.sentences[0].tokens[i].words[0].text + " "
    #         elif doc.sentences[0].tokens[i].words[0].upos == "PART" or doc.sentences[0].tokens[i].words[0].upos == "ADV":
    #             finalText = finalText + doc.sentences[0].tokens[i].words[0].text + " "
    #         i += 1
    #     text = finalText

    return text

df['Cleaned_text'] = df['Review'].progress_apply(preprocess_text)

X = df['Cleaned_text']
y = df['Sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Pongo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
100%|██████████| 772/772 [00:00<00:00, 10961.68it/s]


In [15]:
vectorizer = TfidfVectorizer(max_features=10000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [16]:
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)

MultinomialNB()

In [17]:
y_pred = model.predict(X_test_tfidf)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    Negative       0.96      0.98      0.97        48
     Neutral       0.95      0.98      0.97        57
    Positive       0.98      0.92      0.95        50

    accuracy                           0.96       155
   macro avg       0.96      0.96      0.96       155
weighted avg       0.96      0.96      0.96       155



In [44]:
hypo = model.predict(vectorizer.transform(["bagus"]))
print(hypo)

['Positive']


In [ ]:
dump(model, "model.joblib")
dump(vectorizer, 'vectorizer.pkl')

['model.joblib']

In [19]:
from joblib import load

loaded_model = load('model.joblib')
loaded_vectorizer = load("vectorizer.pkl")

In [20]:
X_test = ["bagus", "jelek", "baik", "buruk"] 
textVec = loaded_vectorizer.transform(X_test)

predictions = loaded_model.predict(textVec).tolist()
print(predictions)

['Positive', 'Negative', 'Neutral', 'Neutral']
